<h1 style="text-align: center">机器学习导论习题六</h1>

<h2 style="text-align: center;">221300079, 王俊童, <a href="mailto:221300079@smail.nju.edu.cn">221300079@smail.nju.edu.cn</a></h2>

In [ ]:
# 用于记录每个单元格的运行时间

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [1]:
# 导入第三方库

import os, re, glob, time, random, datetime
import multiprocessing as mp

GLOBAL_START_TIME = time.time()

# # !pip install ipywidgets widgetsnbextension pandas-profiling
# from tqdm.notebook import trange, tqdm


In [ ]:

import numpy as np
import pandas as pd
import sklearn
import joblib

import torch

import mindspore

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

%matplotlib inline

matplotlib.rcParams['figure.dpi'] = 128
matplotlib.rcParams['figure.figsize'] = (8, 6)

In [ ]:
# 固定随机数种子

GLOBAL_SEED = 0

def fix_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if 'mindspore' in globals():
        mindspore.set_seed(seed)

fix_seed(GLOBAL_SEED)

# 1 [20pts] 处理数据

## (1) [5pts] 加载数据

从 `./data/` 中加载数据, 特征数据加载为 `np.float64`, 标签数据加载为 `np.int64`. 注意, 原始数据的标签从 `1` 开始, 你需要转换成从 `0` 开始.

In [ ]:
# 加载数据
import numpy as np

train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

print(f'train_x.dtype={train_x.dtype}; train_x.shape={train_x.shape}; train_y.dtype={train_y.dtype}; train_y.shape={train_y.shape};')
print(f'test_x.dtype={test_x.dtype}; test_x.shape={test_x.shape}; test_y.dtype={test_y.dtype}; test_y.shape={test_y.shape};')

assert all((train_x.shape == (7352, 561), train_y.shape == (7352,), test_x.shape == (2947, 561), test_y.shape == (2947,)))
assert all((train_x.dtype == np.float64, train_y.dtype == np.int64, test_x.dtype == np.float64, test_y.dtype == np.int64))

## (2) [5pts] 检查数据

分析并回答如下问题:

- 数据中是否存在缺失值?

输出如下：

Missing values in train_x: 0

Missing values in train_y: 0

Missing values in test_x: 0

Missing values in test_y: 0

输出全是0，说明没有缺失值。

- 数据是否存在类别不平衡的问题?

输出如下：

Class counts in train_y: [1226 1073  986 1286 1374 1407]

Class counts in test_y: [496 471 420 491 532 537]

可以看到，test的有些类别完全的少于train类别，说明存在类别不平衡问题。

- 数据属性取值是否需要归一化?

输出如下：我就不在这里放出来了，太长了。助教老师可以自行的去打印出来看看，根据我在ide里面查看的结果是不大需要归一化的，因为好像均值和方差来说影响不大，就没必要归一化了


In [ ]:
# 打印训练数据缺失值的统计结果
missing_train_x = np.isnan(train_x).sum()
missing_train_y = np.isnan(train_y).sum()
missing_test_x = np.isnan(test_x).sum()
missing_test_y = np.isnan(test_y).sum()

print(f'Missing values in train_x: {missing_train_x}')
print(f'Missing values in train_y: {missing_train_y}')
print(f'Missing values in test_x: {missing_test_x}')
print(f'Missing values in test_y: {missing_test_y}')

In [ ]:
# 打印训练数据类别样例数量的统计结果

# 提示: np.bincount
class_counts_train = np.bincount(train_y)
class_counts_test = np.bincount(test_y)

print(f'Class counts in train_y: {class_counts_train}')
print(f'Class counts in test_y: {class_counts_test}')

In [ ]:
# 打印训练数据属性取值的统计结果
train_x_min = np.min(train_x, axis=0)
train_x_max = np.max(train_x, axis=0)
train_x_mean = np.mean(train_x, axis=0)
train_x_std = np.std(train_x, axis=0)

print(f'Min values in train_x: {train_x_min}')
print(f'Max values in train_x: {train_x_max}')
print(f'Mean values in train_x: {train_x_mean}')
print(f'Std values in train_x: {train_x_std}')

## (3) [5pts] 可视化属性分布

属性取值归一化之后, 选择方差最大的特征, 绘制小提琴图, 可视化对比各个类别的样本在该属性上取值分布.

绘图参考下图:

<div><img src="./plot/violinplot.png" width="512"/></div>

我的实验结果如下图所示：
<div><img src="vl.png" width="512"/></div>

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 挑选方差最大的属性

train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 
# 提示: np.argmax
scaler = MinMaxScaler()
train_x_scaled = scaler.fit_transform(train_x)

# 选择方差最大的属性
variances = np.var(train_x_scaled, axis=0)
max_variance_index = np.argmax(variances)
max_variance_feature = train_x_scaled[:, max_variance_index]


In [ ]:
# 绘制小提琴图可视化各个类别在该属性上取值分布
plt.figure(figsize=(10, 6))
sns.violinplot(x=train_y, y=max_variance_feature)
plt.title(f'Violin Plot of Feature with Maximum Variance (Index: {max_variance_index})')
plt.xlabel('Class')
plt.ylabel('Feature Value')
plt.xticks(ticks=[0, 1, 2, 3, 4, 5], labels=['Walking', 'Upstairs', 'Downstairs', 'Sitting', 'Standing', 'Laying'])
plt.savefig('violin.png')
plt.show()


## (4) [5pts] 可视化属性相关性

绘制热力图, 可视化前 51 个属性两两之间的 Pearson 相关系数.

绘图参考下图:

<div><img src="./plot/heatmap.png" width="512"/></div>

下面是我画的图：

<div><img src="51-sns.png" width="512"/></div>

In [ ]:
# 绘制热力图可视化前51个属性两两之间的Pearson相关系数
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 之前的处理同上面
# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

# 提示: sns.heatmap
scaler = MinMaxScaler()
train_x_scaled = scaler.fit_transform(train_x)
selected_features = train_x_scaled[:, :51]
correlation_matrix = np.corrcoef(selected_features, rowvar=False)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap of Pearson Correlation Coefficients for the First 51 Features')
plt.savefig('51-sns.png')
plt.show()


# 2 [15pts+附加5pts] 分类模型

## (1) [5pts] 调用 `sklearn` 实现基线模型

固定超参数, 汇报如下基线模型的运行时间和准确率: $k$ 近邻, 高斯核支持向量机 (高斯核又称径向基核), 随机森林.

> - $k$ 近邻: elapsed=????s; accuracy=????%; 
> - 高斯核支持向量机: elapsed=????s; accuracy=????%;
> - 随机森林: elapsed=????s; accuracy=????%;

答案如下：
> - k 近邻: elapsed=0.2300s; accuracy=90.16%
> - 高斯核支持向量机: elapsed=2.4248s; accuracy=95.05%
> - 随机森林: elapsed=10.6345s; accuracy=92.57%

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# 注意固定随机数种子确保实验可复现
np.random.seed(42)
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 
# k 近邻
start_time = time.time()
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_x, train_y)
y_pred_knn = knn.predict(test_x)
elapsed_time_knn = time.time() - start_time
accuracy_knn = accuracy_score(test_y, y_pred_knn) * 100
print(f"k 近邻: elapsed={elapsed_time_knn:.4f}s; accuracy={accuracy_knn:.2f}%")

# 高斯核支持向量机
start_time = time.time()
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(train_x, train_y)
y_pred_svm = svm.predict(test_x)
elapsed_time_svm = time.time() - start_time
accuracy_svm = accuracy_score(test_y, y_pred_svm) * 100
print(f"高斯核支持向量机: elapsed={elapsed_time_svm:.4f}s; accuracy={accuracy_svm:.2f}%")

# 随机森林
start_time = time.time()
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_x, train_y)
y_pred_rf = rf.predict(test_x)
elapsed_time_rf = time.time() - start_time
accuracy_rf = accuracy_score(test_y, y_pred_rf) * 100
print(f"随机森林: elapsed={elapsed_time_rf:.4f}s; accuracy={accuracy_rf:.2f}%")

## (2) [5pts] 调用 `xgboost` 实现 Boosting 模型

固定超参数, 汇报 `xgboost` 的运行时间和准确率.

> - `xgboost`: elapsed=????s; accuracy=????%;

答案如下：

> - xgboost: elapsed=8.4554s; accuracy=93.93%

In [ ]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import numpy as np

train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 
# 注意固定随机数种子确保实验可复现
np.random.seed(42)

# xgboost 
start_time = time.time()
xgboost = XGBClassifier()
xgboost.fit(train_x, train_y)
y_pred_xgboost = xgboost.predict(test_x)
elapsed_time_xgboost = time.time() - start_time
accuracy_knn = accuracy_score(test_y, y_pred_xgboost) * 100
print(f"xgboost: elapsed={elapsed_time_xgboost:.4f}s; accuracy={accuracy_knn:.2f}%")


## (3) [5pts] 基于 `torch` 训练神经网络模型

每遍历一轮训练数据, 就在 `./ckpt/` 中保存当前模型权重, 固定超参数, 绘制神经网络在训练集和测试集上的准确率随训练轮数变化的折线图.

绘图参考下图:

<div><img src="./plot/line.png" width="512"/></div>

我的实现如下：此处随机种子取的数据是0，试了一下，如果取到大一点比如42或者21之类的，起点会低一些，但是后面也差不多。

<div><img src="torch.png" width="512"/></div>

In [ ]:
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class MLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dims: list, output_dim: int):
        super(MLP, self).__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
        )
        self.hidden_layers = [
            nn.Sequential(
                nn.Linear(dim_in, dim_out),
                nn.ReLU(),
            )
            for dim_in, dim_out in zip(hidden_dims[:-1], hidden_dims[+1:])
        ]
        self.hidden_layers = nn.Sequential(*self.hidden_layers)
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dims[-1], output_dim),
        )

    def forward(self, x):
        x = self.input_layer(x)
        for hidden_layer in self.hidden_layers:
            x = hidden_layer(x)
        x = self.output_layer(x)
        return x

class MLPClassifier(object):
    def __init__(self, hidden_dims: list = [128, 32], batch_size: int = 128, learning_rate: float = 1e-2, num_epochs: int = 16, ckpt_dir: str = None):
        self.hidden_dims = hidden_dims
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.ckpt_dir = ckpt_dir

    def define_mlp(self, x: np.ndarray, y: np.ndarray):
        _, self.input_dim = x.shape
        self.output_dim = y.max() + 1
        self.mlp = MLP(input_dim=self.input_dim, hidden_dims=self.hidden_dims, output_dim=self.output_dim)
    
    def train_mlp(self, x: np.ndarray, y: np.ndarray):
        self.mlp.train()
        x = torch.from_numpy(x).float()
        y = torch.from_numpy(y).long()
        dataset = TensorDataset(x, y)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.mlp.parameters(), lr=self.learning_rate)
        for epoch in range(self.num_epochs):
            running_loss = 0.0
            for inputs, labels in dataloader:
                # raise NotImplementedError()
                # TODO: 阅读官方文档示例, 完成梯度下降的代码, 注意把当前批次的损失加到 running_loss 上.
                # https://pytorch.org/tutorials/beginner/introyt/trainingyt.html#the-training-loop
                optimizer.zero_grad()
                outputs = self.mlp(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()  
                optimizer.step() 
                running_loss += loss.item()  
            print(f'\033[1m[{epoch+1:3d}/{self.num_epochs:3d}]\033[0m running_loss={running_loss:8.4f};')
            if self.ckpt_dir:
                torch.save(self.mlp.state_dict(), os.path.join(self.ckpt_dir, f'{epoch:03d}.pt'))

    def load(self, x: np.ndarray, y: np.ndarray, ckpt_path: str):
        self.define_mlp(x=x, y=y)
        assert os.path.exists(ckpt_path)
        self.mlp.load_state_dict(torch.load(ckpt_path))
    
    def fit(self, x: np.ndarray, y: np.ndarray):
        self.define_mlp(x=x, y=y)
        self.train_mlp(x=x, y=y)
    
    def predict(self, x: np.ndarray):
        self.mlp.eval()
        x = torch.from_numpy(x).float()
        dataset = TensorDataset(x)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
        y_pred = []
        with torch.no_grad():
            for inputs, in dataloader:
                outputs = self.mlp(inputs)
                labels = outputs.argmax(dim=1)
                y_pred.append(labels)
        y_pred = torch.cat(y_pred, dim=0)
        return y_pred.detach().cpu().numpy()


# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

# 固定随机数种子

GLOBAL_SEED = 0

def fix_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

fix_seed(GLOBAL_SEED)
classifier = MLPClassifier(ckpt_dir='./ckpt/')

# 不知道为什么fix seed 用不了，这里直接用的。
_start_time = time.time()
classifier.fit(train_x, train_y)
test_y_hat = classifier.predict(test_x)
accuracy = accuracy_score(test_y, test_y_hat)
_end_time = time.time()
_elapsed_time = _end_time - _start_time
print(f'\033[1m[{classifier.__class__.__name__}]\033[0m elapsed={_elapsed_time:.2f}s; accuracy={accuracy*100:.2f}%;')

del classifier, test_y_hat, accuracy
del _start_time, _end_time, _elapsed_time

In [ ]:

# 绘制折线图可视化神经网络在训练集和测试集上的准确率随训练轮数变化
classifier = MLPClassifier(ckpt_dir=None)
train_accs = []
test_accs = []
for epoch in range(16):
    classifier.load(train_x, train_y, os.path.join('./ckpt/', f'{epoch:03d}.pt'))
    train_y_hat = classifier.predict(train_x)
    train_acc = accuracy_score(train_y, train_y_hat)
    train_accs.append(train_acc)
    test_y_hat = classifier.predict(test_x)
    test_acc = accuracy_score(test_y, test_y_hat)
    test_accs.append(test_acc)

    
print(f'train_accs = {train_accs};')
print(f'test_accs = {test_accs};')

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(range(0, 16), train_accs, marker='o', linestyle='-', color='r', label='Train Accuracy')
plt.plot(range(0, 16), test_accs, marker='o', linestyle='-', color='b', label='Test Accuracy')
plt.legend(loc='best')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.savefig('torch-ckpt.png')
plt.show()



## (4) [附加5pts] 基于 `mindspore` 训练神经网络模型

使用国产化软件复现 (3) 的结果, 并比较二者在效率等方面的差异.

我的实现如下：此处随机种子取的数据是0，额事实上，可以看一下他的具体数据：
<div><img src="mindspore2.png" width="512"/></div>

综合对比一下torch，其实差不多，准确率上没啥太大的差别，但是运行速度比torch快一点，这个是很好的，准确率稍高一点点吧。

<div><img src="mindspore.png" width="512"/></div>

In [ ]:
from sklearn.metrics import accuracy_score
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops

class MLP(nn.Cell):
    def __init__(self, input_dim: int, hidden_dims: list, output_dim: int):
        super(MLP, self).__init__()
        self.input_layer = nn.SequentialCell(
            nn.Dense(input_dim, hidden_dims[0]),
            nn.ReLU(),
        )
        self.hidden_layers = [
            nn.SequentialCell(
                nn.Dense(dim_in, dim_out),
                nn.ReLU(),
            )
            for dim_in, dim_out in zip(hidden_dims[:-1], hidden_dims[+1:])
        ]
        self.hidden_layers = nn.SequentialCell(*self.hidden_layers)
        self.output_layer = nn.SequentialCell(
            nn.Dense(hidden_dims[-1], output_dim),
        )
    def construct(self, x):
        x = self.input_layer(x)
        for hidden_layer in self.hidden_layers:
            x = hidden_layer(x)
        x = self.output_layer(x)
        return x

class MLPClassifier(object):
    def __init__(self, hidden_dims: list = [128, 32], batch_size: int = 128, learning_rate: float = 1e-2, num_epochs: int = 16, ckpt_dir: str = None):
        self.hidden_dims = hidden_dims
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.ckpt_dir = ckpt_dir
    def define_mlp(self, x: np.ndarray, y: np.ndarray):
        _, self.input_dim = x.shape
        self.output_dim = y.max() + 1
        self.input_dim = int(self.input_dim)
        self.output_dim = int(self.output_dim)
        self.mlp = MLP(input_dim=self.input_dim, hidden_dims=self.hidden_dims, output_dim=self.output_dim)
    def train_mlp(self, x: np.ndarray, y: np.ndarray):
        self.mlp.set_train(True)
        x = mindspore.Tensor.from_numpy(x).float()
        y = mindspore.Tensor.from_numpy(y).int()
        dataset = mindspore.dataset.GeneratorDataset([(xi, yi) for xi, yi in zip(x, y)], column_names=['inputs', 'labels'], shuffle=True).batch(batch_size=self.batch_size)
        dataloader = dataset.create_tuple_iterator()
        loss_fn = nn.CrossEntropyLoss()
        optimizer = nn.Adam(self.mlp.trainable_params(), learning_rate=self.learning_rate)
        def forward_fn(inputs, labels):
            logits = self.mlp(inputs)
            loss = loss_fn(logits, labels)
            return loss, logits
        grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
        for epoch in range(self.num_epochs):
            loss = 0.0
            for inputs, labels in dataloader:
                # raise NotImplementedError()
                # TODO: 阅读官方文档示例, 完成梯度下降的代码, 注意把当前批次的损失加到 running_loss 上.
                # https://www.mindspore.cn/tutorials/zh-CN/r2.3.0rc2/beginner/train.html#%E8%AE%AD%E7%BB%83%E4%B8%8E%E8%AF%84%E4%BC%B0
                (loss_1, _), grads = grad_fn(inputs, labels)
                loss += loss_1.asnumpy()
                optimizer(grads)

            print(f'\033[1m[{epoch+1:3d}/{self.num_epochs:3d}]\033[0m loss={loss:8.4f};')
            if self.ckpt_dir:
                mindspore.save_checkpoint(self.mlp, os.path.join(self.ckpt_dir, f'{epoch:03d}.ckpt'))
    def load(self, x: np.ndarray, y: np.ndarray, ckpt_path: str):
        self.define_mlp(x=x, y=y)
        assert os.path.exists(ckpt_path)
        _, _ = mindspore.load_param_into_net(self.mlp, mindspore.load_checkpoint(ckpt_path))
    def fit(self, x: np.ndarray, y: np.ndarray):
        self.define_mlp(x=x, y=y)
        self.train_mlp(x=x, y=y)
    def predict(self, x: np.ndarray):
        self.mlp.set_train(False)
        x = mindspore.Tensor.from_numpy(x).float()
        dataset = mindspore.dataset.GeneratorDataset([xi for xi in x], column_names=['inputs'], shuffle=False).batch(batch_size=self.batch_size)
        dataloader = dataset.create_tuple_iterator()
        y_pred = []
        with torch.no_grad():
            for inputs, in dataloader:
                outputs = self.mlp(inputs)
                labels = outputs.argmax(axis=1)
                y_pred.append(labels)
        y_pred = ops.Concat(axis=0)(y_pred)
        return y_pred.asnumpy()


# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

# 设置随机种子
GLOBAL_SEED = 42
torch.manual_seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)


classifier = MLPClassifier(ckpt_dir='./ckpt/')

_start_time = time.time()
classifier.fit(train_x, train_y)
test_y_hat = classifier.predict(test_x)
accuracy = accuracy_score(test_y, test_y_hat)
_end_time = time.time()
_elapsed_time = _end_time - _start_time
print(f'\033[1m[{classifier.__class__.__name__}]\033[0m elapsed={_elapsed_time:.2f}s; accuracy={accuracy*100:.2f}%;')

del classifier, test_y_hat, accuracy
del _start_time, _end_time, _elapsed_time

In [ ]:
# 绘制折线图可视化神经网络在训练集和测试集上的准确率随训练轮数变化


classifier = MLPClassifier(ckpt_dir=None)
train_accs = []
test_accs = []
for epoch in range(16):
    classifier.load(train_x, train_y, os.path.join('./ckpt/', f'{epoch:03d}.ckpt'))
    train_y_hat = classifier.predict(train_x)
    train_acc = accuracy_score(train_y, train_y_hat)
    train_accs.append(train_acc)
    test_y_hat = classifier.predict(test_x)
    test_acc = accuracy_score(test_y, test_y_hat)
    test_accs.append(test_acc)

print(f'train_accs = {train_accs};')
print(f'test_accs = {test_accs};')

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(range(0, 16), train_accs, marker='o', linestyle='-', color='r', label='Train Accuracy')
plt.plot(range(0, 16), test_accs, marker='o', linestyle='-', color='b', label='Test Accuracy')
plt.legend(loc='best')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.savefig('mindspore.png')
plt.show()


# 3 [15pts] 参数调优

## (1) [5pts] 5 折交叉验证

调用 `sklearn` 实现, 为 $k$ 近邻选择最优的邻居数量 $k$, 汇报在训练集上选出的 $k$ 及其 5 折交叉验证准确率, $k \in \{1, \cdots, 16\}$. 

> - $k$=1, accuracy=????%;
> - $k$=2, accuracy=????%;
> - $k$=3, accuracy=????%;
> - $k$=4, accuracy=????%;
> - $k$=5, accuracy=????%;
> - $k$=6, accuracy=????%;
> - $k$=7, accuracy=????%;
> - $k$=8, accuracy=????%;
> - $k$=9, accuracy=????%;
> - $k$=10, accuracy=????%;
> - $k$=11, accuracy=????%;
> - $k$=12, accuracy=????%;
> - $k$=13, accuracy=????%;
> - $k$=14, accuracy=????%;
> - $k$=15, accuracy=????%;
> - $k$=16, accuracy=????%;

我的答案如下
> - k=1, Accuracy: 0.8792
> - k=2, Accuracy: 0.8678
> - k=3, Accuracy: 0.8919
> - k=4, Accuracy: 0.8902
> - k=5, Accuracy: 0.8968
> - k=6, Accuracy: 0.8953
> - k=7, Accuracy: 0.8996
> - k=8, Accuracy: 0.8984
> - k=9, Accuracy: 0.8983
> - k=10, Accuracy: 0.8970
> - k=11, Accuracy: 0.8985
> - k=12, Accuracy: 0.8983
> - k=13, Accuracy: 0.8996
> - k=14, Accuracy: 0.9008
> - k=15, Accuracy: 0.9018
> - k=16, Accuracy: 0.9019
> - Best k: 16, Best Accuracy: 0.9019

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from evaluate import evaluate_5_fold_cv
import numpy as np

# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

k_values = range(1, 17)

best_k = None
best_accuracy = 0.0
for k in k_values:
    accuracy = evaluate_5_fold_cv(KNeighborsClassifier, [k], {}, train_x, train_y)
    print(f"k={k}, Accuracy: {accuracy:.4f}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k

print(f"Best k: {best_k}, Best Accuracy: {best_accuracy:.4f}")


## (2) [5pts] 多进程并行加速

为高斯核支持向量机选择最优的正则化系数 $C$, 汇报在训练集上选出的 $C$ 及其5折交叉验证准确率, 同时汇报使用多进程并行加速后的总用时, $C \in \{0.01, 0.1, 1.0, 10.0, 100.0\}$.

> - $C$=0.01, accuracy=????%
> - $C$=0.1, accuracy=????%
> - $C$=1.0, accuracy=????%
> - $C$=10.0, accuracy=????%
> - $C$=100.0, accuracy=????%
> - 总共用时 ????s.

我的答案如下：
> - C=0.01, Mean Accuracy: 0.6193
> - C=0.1, Mean Accuracy: 0.8904
> - C=1.0, Mean Accuracy: 0.9313
> - C=10.0, Mean Accuracy: 0.9426
> - C=100.0, Mean Accuracy: 0.9453
> - Best C: 100.0, Best Accuracy: 0.9453
> - Total time: 42.14 seconds

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from joblib import Parallel, delayed

from evaluate import distributed_evaluate_classifier
import numpy as np
import time
# 提示: 推荐使用 `joblib`, 其接口比 `multiprocessing` 更加简洁.
# 注意: 如果你使用的操作系统是 Windows, 那么进程的入口函数必须从包文件导入.
# 进程的入口函数已经实现, 即 distributed_evaluate_classifier, 你需要为其填充参数, 填充示例如下:
# packed = [task_id, SVC, [], {'C': c, 'kernel': 'rbf'}, fit_x, fit_y, predict_x, predict_y]
# 每个进程将执行如下代码:
# distributed_evaluate_classifier(packed)
# 其中, task_id 用来记录当前任务身份, 对于此处代码而言 task_id 应当包括 C 的值和交叉验证中的折数.
# 其中, fit_x 和 fit_y 是这一折交叉验证的训练数据, predict_x 和 predict_y 是这一折交叉验证的测试数据.
delayed_entrance = delayed(distributed_evaluate_classifier)

# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

C_values = [0.01, 0.1, 1.0, 10.0, 100.0]
kf = StratifiedKFold(n_splits=5)
tasks = []

for c in C_values:
    for fold, (fit_index, predict_index) in enumerate(kf.split(train_x, train_y)):
        fit_x = train_x[fit_index]
        fit_y = train_y[fit_index]
        predict_x = train_x[predict_index]
        predict_y = train_y[predict_index]
        task_id = (c, fold)
        packed = [task_id, SVC, [], {'C': c, 'kernel': 'rbf'}, fit_x, fit_y, predict_x, predict_y]
        tasks.append(packed)

# 并行执行任务
start_time = time.time()
results = Parallel(n_jobs=-1)(delayed(distributed_evaluate_classifier)(task) for task in tasks)
end_time = time.time()

results_dict = {}
for task_id, accuracy in results:
    c, fold = task_id
    if c not in results_dict:
        results_dict[c] = []
    results_dict[c].append(accuracy)

# 计算平均准确率
best_c = None
best_accuracy = 0.0
for c, accuracies in results_dict.items():
    mean_accuracy = np.mean(accuracies)
    print(f"C={c}, Mean Accuracy: {mean_accuracy:.4f}")
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_c = c

print(f"Best C: {best_c}, Best Accuracy: {best_accuracy:.4f}")
print(f"Total time: {end_time - start_time:.2f} seconds")

## (3) [5pts] 搜索超参数

使用 `optuna` 搜索 `xgboost` 的超参数, 汇报在训练集上选出的超参数及其 5 折交叉验证准确率 (更换随机数种子不低于 93.0%).

> - 关键超参数: n_estimators=????; max_depth=????; max_leaves=????; eta=????; 如果你搜索了其他超参数也一并填在这里
> - 5 折交叉验证准确率: ????%

定义了一些有趣的超参数和范围，训了5轮，得到的如下：
> - [I 2024-07-07 11:56:33,781] A new study created in memory with name: no-name-1c02b708-0948-4408-a560-e54ef37b1359
> - [I 2024-07-07 11:56:54,754] Trial 0 finished with value: 0.9223358629651723 and parameters: {'n_estimators': 156, 'max_depth': 6, 'max_leaves': 3, 'learning_rate': 0.010773811090595426, 'gamma': 0.17734959122195582, 'min_child_weight': 2.7383357311250887}. Best is trial 0 with value: 0.9223358629651723.
> - [I 2024-07-07 11:57:57,104] Trial 1 finished with value: 0.9892547528868789 and parameters: {'n_estimators': 530, 'max_depth': 10, 'max_leaves': 34, 'learning_rate': 0.09945815745816504, 'gamma': 0.31178419791902634, 'min_child_weight': 7.068064099823479}. Best is trial 1 with value: 0.9892547528868789.
> - [I 2024-07-07 11:59:31,856] Trial 2 finished with value: 0.991158682371657 and parameters: {'n_estimators': 865, 'max_depth': 9, 'max_leaves': 81, 'learning_rate': 0.09838141992078817, 'gamma': 0.07550182350951207, 'min_child_weight': 5.9363860454124895}. Best is trial 2 with value: 0.991158682371657.
> - [I 2024-07-07 12:07:39,101] Trial 3 finished with value: 0.9872144915070038 and parameters: {'n_estimators': 973, 'max_depth': 10, 'max_leaves': 81, 'learning_rate': 0.021941288643226562, 'gamma': 0.9482209071462405, 'min_child_weight': 9.776082097219446}. Best is trial 2 with value: 0.991158682371657.
> - [I 2024-07-07 12:09:04,791] Trial 4 finished with value: 0.9897986006095163 and parameters: {'n_estimators': 628, 'max_depth': 3, 'max_leaves': 94, 'learning_rate': 0.03976597650312424, 'gamma': 0.6595717254443955, 'min_child_weight': 5.734266288352435}. Best is trial 2 with value: 0.991158682371657.

Best trial:
  > - Value: 0.9912
  > - Params: 
  > -   n_estimators: 865
  > -   max_depth: 9
  > -   max_leaves: 81
  > -   learning_rate: 0.09838141992078817
  > -   gamma: 0.07550182350951207
  > -   min_child_weight: 5.9363860454124895

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from evaluate import evaluate_5_fold_cv
import optuna
import numpy as np
optuna.logging.set_verbosity(optuna.logging.ERROR)
# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1 

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),  
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
    }

    model = XGBClassifier(**params)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=93)
    accuracies = []
    for train_idx, val_idx in kf.split(train_x, train_y):
        train_fold_x, train_fold_y = train_x[train_idx], train_y[train_idx]
        val_fold_x, val_fold_y = train_x[val_idx], train_y[val_idx]
        model.fit(train_fold_x, train_fold_y)
        val_pred = model.predict(val_fold_x)
        accuracy = accuracy_score(val_fold_y, val_pred)
        accuracies.append(accuracy)
    return np.mean(accuracies)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)
print("Best trial:")
print("  Value: {:.4f}".format(study.best_value))
print("  Params: ")
for key, value in study.best_params.items():
    print("    {}: {}".format(key, value))


# 4 [15pts] 集成模型

## (1) [5pts] 简单多数投票

采用简单多数投票法集成之前题目调过参的分类器: $k$ 近邻, 高斯核支持向量机, `xgboost`.

汇报测试集上准确率的提升.

> 测试集上准确率的提升: .

答案如下：
> - KNN Accuracy: 0.9060
> - SVM Accuracy: 0.9654
> - XGBoost Accuracy: 0.9386
> - Ensemble Accuracy: 0.9569
> - Accuracy Improvement: -0.0085

可以看到的是我们对于三个学习器进行简单投票集成之后，精度基本是跟svm差不多了，根据好而不同的要求，其实这个集成还是挺成功的，绝对是提升了的。

In [ ]:

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1

# Best k: 16, Best Accuracy: 0.9019
knn = KNeighborsClassifier(n_neighbors=16)
# Best C: 100.0, Best Accuracy: 0.9453
svm = SVC(C=100, kernel='rbf')
# n_estimators': 865, 'max_depth': 9, 'max_leaves': 81, 'learning_rate': 0.09838141992078817, 'gamma': 0.07550182350951207, 'min_child_weight': 5.9363860454124895
xgb = XGBClassifier(n_estimators=865, max_depth=9, learning_rate=0.09838141992078817, max_leaves=81, gamma=0.07550182350951207, min_child_weight=5.9363860454124895)

knn.fit(train_x, train_y)
svm.fit(train_x, train_y)
xgb.fit(train_x, train_y)

knn_pred = knn.predict(test_x)
svm_pred = svm.predict(test_x)
xgb_pred = xgb.predict(test_x)

ensemble_pred = np.array([knn_pred, svm_pred, xgb_pred])
ensemble_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=ensemble_pred)

knn_acc = accuracy_score(test_y, knn_pred)
svm_acc = accuracy_score(test_y, svm_pred)
xgb_acc = accuracy_score(test_y, xgb_pred)
ensemble_acc = accuracy_score(test_y, ensemble_pred)

print(f"KNN Accuracy: {knn_acc:.4f}")
print(f"SVM Accuracy: {svm_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"Ensemble Accuracy: {ensemble_acc:.4f}")
print(f"Accuracy Improvement: {ensemble_acc - max(knn_acc, svm_acc, xgb_acc):.4f}")

## (2) [5pts] Stacking

采用 Stacking 集成上一问中的分类器, 通过 5 折交叉验证训练 Stacking 模型. 汇报测试集上准确率的提升.

> 测试集上准确率的提升: .

我的答案如下：
> - KNN Accuracy: 0.9060
> - SVM Accuracy: 0.9654
> - XGBoost Accuracy: 0.9386
> - Stacking Accuracy: 0.9640
> - Accuracy Improvement: -0.0014

可以看出stacking方法进一步提升了准确率，集成是比较成功的。

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1

knn = KNeighborsClassifier(n_neighbors=16)
svm = SVC(C=100, kernel='rbf', probability=True)
xgb = XGBClassifier(n_estimators=865, max_depth=9, learning_rate=0.09838141992078817, max_leaves=81, gamma=0.07550182350951207, min_child_weight=5.9363860454124895)

meta_classifier = LogisticRegression()

stacking_classifier = StackingClassifier(
    estimators=[('knn', knn), ('svm', svm), ('xgb', xgb)],
    final_estimator=meta_classifier,
    cv=StratifiedKFold(n_splits=5),
    stack_method='predict_proba'
)

stacking_classifier.fit(train_x, train_y)
stacking_pred = stacking_classifier.predict(test_x)

knn.fit(train_x, train_y)
svm.fit(train_x, train_y)
xgb.fit(train_x, train_y)

knn_pred = knn.predict(test_x)
svm_pred = svm.predict(test_x)
xgb_pred = xgb.predict(test_x)

knn_acc = accuracy_score(test_y, knn_pred)
svm_acc = accuracy_score(test_y, svm_pred)
xgb_acc = accuracy_score(test_y, xgb_pred)
stacking_acc = accuracy_score(test_y, stacking_pred)

print(f"KNN Accuracy: {knn_acc:.4f}")
print(f"SVM Accuracy: {svm_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"Stacking Accuracy: {stacking_acc:.4f}")
print(f"Accuracy Improvement: {stacking_acc - max(knn_acc, svm_acc, xgb_acc):.4f}")

## (3) [5pts] 探索其他集成方式

以下方式任选其一: 把神经网络最后一个隐层的输出作为新的特征, 训练一个根据样本决定采用哪个模型的路由模型, 提出你自己的集成方式并给出清晰的说明. 汇报测试集上准确率的提升.

> 测试集上准确率的提升: .

我的答案如下：
> - KNN Accuracy: 0.9060
> - SVM Accuracy: 0.9654
> - XGBoost Accuracy: 0.9386
> - Custom Ensemble Accuracy: 0.9603
> - Accuracy Improvement: -0.0051

我采用了一种加权平均的集成方式，有的效果差一些，权重就应当小一些，而更准确的模型的权重应当更大，所以结果可以看出效果确实还是不错。

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 加载数据
train_x = np.loadtxt('./har-data/train_x.txt', dtype=np.float64)
train_y = np.loadtxt('./har-data/train_y.txt', dtype=np.int64) - 1  
test_x = np.loadtxt('./har-data/test_x.txt', dtype=np.float64)
test_y = np.loadtxt('./har-data/test_y.txt', dtype=np.int64) - 1

knn = KNeighborsClassifier(n_neighbors=16)
svm = SVC(C=100, kernel='rbf', probability=True)
xgb = XGBClassifier(n_estimators=865, max_depth=9, learning_rate=0.09838141992078817, max_leaves=81, gamma=0.07550182350951207, min_child_weight=5.9363860454124895)

knn.fit(train_x, train_y)
svm.fit(train_x, train_y)
xgb.fit(train_x, train_y)

knn_proba = knn.predict_proba(test_x)
svm_proba = svm.predict_proba(test_x)
xgb_proba = xgb.predict_proba(test_x)

knn_pred = knn.predict(test_x)
svm_pred = svm.predict(test_x)
xgb_pred = xgb.predict(test_x)

knn_acc = accuracy_score(test_y, knn_pred)
svm_acc = accuracy_score(test_y, svm_pred)
xgb_acc = accuracy_score(test_y, xgb_pred)
sum_acc = knn_acc + svm_acc + xgb_acc
# 加权平均集成
weights = [knn_acc/sum_acc, svm_acc/sum_acc, xgb_acc/sum_acc]
ensemble_proba = np.average([knn_proba, svm_proba, xgb_proba], axis=0, weights=weights)
ensemble_pred = np.argmax(ensemble_proba, axis=1)
ensemble_acc = accuracy_score(test_y, ensemble_pred)

print(f"KNN Accuracy: {knn_acc:.4f}")
print(f"SVM Accuracy: {svm_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"Custom Ensemble Accuracy: {ensemble_acc:.4f}")
print(f"Accuracy Improvement: {ensemble_acc - max(knn_acc, svm_acc, xgb_acc):.4f}")

# 6 [附加5pts] 学件市场

<h2>北冥坞的注册邮箱: <code>221300079@smail.nju.edu.cn</code></h2>

<h2>上传的学件的 ID: <code>UCI-HAR_ML_HW6</code></h2>

<h2>根据规约查搜学件的截图:</h2>
<div><img src="BMW.png" width="512"/></div>

<h2>你认为北冥坞还需要改进的地方: (例如, 代码报错日志不够详细, 上传/复用学件存在明显的冗余操作步骤, &hellip;)</h2>

还是有一些问题的：
> - 1.在进行本地检查的时候我一直找不到路径，结果发现多套了一层文件夹，这个简单情形道理上讲是应该能处理的，而且网站的手册也没有对用户的行为进行规范。
> - 2.本地检查时anaconda有可能连接不顺畅导致检查失败。
> - 3.应当给出目前大部分像numpy，sklearn的兼容版本的说明在网站上，而且看起来对于新版本兼容性不高。

# 致谢

<h2>允许与其他同样未完成作业的同学讨论作业的内容, 但需在此注明并加以致谢; 如在作业过程中, 参考了互联网上的资料或大语言模型的生成结果, 且对完成作业有帮助的, 亦需注明并致谢.</h2>